#Data_Augmentation_Methods:

### This notebook contains three different state-of-the-art data augmentation techniques:
1. Easy Data Augmentation Techniques for Boosting Performance on Text Classification Tasks
2. Backtranslation using transformer models
3. Feeding surroundings word to BERT language model to find out the most suitlabe word for augmentation (nlpaug)

## Method 1. EDA: Easy Data Augmentation Techniques for Boosting Performance on Text Classification Tasks

Reference: https://arxiv.org/abs/1901.11196

1. Synonym Replacement (SR): Randomly choose n words from the sentence that are not stop words. Replace each of these words with one of its synonyms chosen at random.

2. Random Insertion (RI): Find a random synonym of a random word in the sentence that is not a stop word. Insert that synonym into a random position in the sentence. Do this n times.
3. Random Swap (RS): Randomly choose two words in the sentence and swap their positions. Do this n times.
4. Random Deletion (RD): For each word in the sentence, randomly remove it with probability p.

In [ ]:
!pip install -U nltk

     |████████████████████████████████| 1.5MB 11.1MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import nltk; nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
%cd /content/drive/MyDrive/Capstone-project_Bhavika/data_aug

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/projects/Capstone_Project/Checkpoint_6/processed_data

/content/drive/MyDrive/projects/Capstone_Project/Checkpoint_6/processed_data


In [ ]:
!python code/augment.py --input=/content/drive/MyDrive/projects/Capstone_Project/Checkpoint_6/processed_data/labeled_1.pkl --num_aug=10 --alpha_sr=0.05 --alpha_rd=0.05 --alpha_ri=0.05 --alpha_rs=0.05

python3: can't open file 'code/augment.py': [Errno 2] No such file or directory


In [ ]:
lines = open("/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/eda_nlp/data/train_Q2.tsv", 'r').readlines()
counter =0
for i, line in enumerate(lines):
  parts = line.split(" ",1)
  label = parts[0]
  sentence = parts[1]
  if len(parts[1])==0:
    counter = counter+1
  #print(len(parts[1]),"------",parts[0],"------",parts[1])
print(counter)

# Method 2. Backtranslation

Reference: https://github.com/makcedward/nlpaug

In [ ]:
%cd

/root


In [ ]:
!pip install numpy requests nlpaug

     |████████████████████████████████| 399kB 10.9MB/s 


In [ ]:
!pip install torch>=1.6.0 fairseq>=0.9.0 sacremoses>=0.0.43 fastBPE>=0.1.0

In [ ]:
import pandas as pd

In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [ ]:
# This is just a test
import nlpaug.augmenter.word as naw

text = 'The quick brown fox jumped over the lazy dog'
back_translation_aug = naw.BackTranslationAug(
    from_model_name='transformer.wmt19.en-de', 
    to_model_name='transformer.wmt19.de-en'
)
back_translation_aug.augment(text)

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip
100%|██████████| 11946275315/11946275315 [08:07<00:00, 24517982.52B/s]
Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master
100%|██████████| 11958904958/11958904958 [08:26<00:00, 23633844.79B/s]


'The speedy brown fox jumped over the lazy dog'

In [ ]:
# Let's try to not cry here
input_data = "/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/backtranslation/data/train_Q2.tsv"
output_file = "/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/backtranslation/data/bt_train_Q2.tsv"
writer = open(output_file, 'w')
lines = open(input_data, 'r').readlines()


for i, line in enumerate(lines):
  # parts = line[:-1].split('\t')
  parts = line.split(" ",1)
  label = parts[0]
  sentence = parts[1]
  writer.write(label + "\t" + back_translation_aug.augment(parts[1]) + '\n')
  df.append({'label':[label],'sentence':[back_translation_aug.augment(parts[1])]},ignore_index=True)
  print(i)

writer.close()

In [ ]:
%%bash
pip install fastBPE regex requests sacremoses subword_nmt

In [ ]:
# List available models
#print(torch.hub.list('pytorch/fairseq'))  # [..., 'transformer.wmt19.en-de.single_model', ... ]
# Load a transformer trained on WMT'16 En-De
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')


data_path = '/content/drive/MyDrive/projects/Capstone_Project/Checkpoint_6/processed_data/'

with open(data_path + 'labeled_1.pkl', 'rb') as f:
    train_labeled_data = pickle.load(f)

num_sample_sen = 2
cnt = 0
train_unlabeled_data_aug = {}

for key, value in tqdm(train_labeled_data.items(), ncols=50, desc="Iteration:"):
    new_value = []

    for i in range(num_sample_sen):
        v = de2en.translate(en2de.translate(value, sampling = True, temperature = 0.8),
                                    sampling = True, temperature = 0.8)
        if cnt % 100 == 0:
            print("***************")
            print("org: ", value)
            print("new: ", v)
        new_value.append(v)
    train_unlabeled_data_aug[key] = new_value
    if cnt % 1000 == 0:
        with open(data_path + 'train_labeled_data_bt.pkl', 'wb') as f:
            assert len(train_unlabeled_data_aug[key]) == num_sample_sen
            pickle.dump(train_unlabeled_data_aug, f)
    cnt += 1

with open(data_path + 'train_labeled_data_bt.pkl', 'wb') as f:
    pickle.dump(train_unlabeled_data_aug, f)

In [ ]:
with open(data_path + 'labeled_2.pkl', 'rb') as g:
    train_labeled_data = pickle.load(g)

num_sample_sen = 2
cnt = 0

for key, value in tqdm(train_labeled_data.items(), ncols=50, desc="Iteration:"):
    new_value = []

    for i in range(num_sample_sen):
        v = de2en.translate(en2de.translate(value, sampling = True, temperature = 0.8),
                                    sampling = True, temperature = 0.8)
        print(key)
        if cnt % 100 == 0:
            print("***************")
            print("Original Value: ", value)
            print("New Value: ", v)
            print("***************")
        new_value.append(v)
    if cnt % 1000 == 0:
        with open(data_path + 'train_labeled_data_bt_2.pkl', 'wb') as f:
            assert len(new_value) == num_sample_sen
            pickle.dump(new_value, f)
    cnt += 1

with open(data_path + 'train_labeled_data_bt_2.pkl', 'wb') as f:
    pickle.dump(new_value, f)

# Method 3: Feeding surroundings word to BERT language model to find out the most suitlabe word for augmentation

Reference: https://github.com/makcedward/nlpaug

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 11.2MB/s 
     |████████████████████████████████| 901kB 37.1MB/s 
     |████████████████████████████████| 3.3MB 28.3MB/s 


In [ ]:
import transformers

In [ ]:
text = "The quick brown fox jumps over the lazy dog"
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)


Original:
The quick brown fox jumps over the lazy dog
Augmented Text:
the quick brown fox jumps both over on the lazy dog


In [ ]:
#pkl file
import pickle

with open('/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/backtranslation/data/labeled_1.pkl', 'rb') as f:
    labeled_1 = pickle.load(f)

with open('/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/backtranslation/data/labeled_2.pkl', 'rb') as t:
    labeled_2 = pickle.load(t)

In [ ]:
labeled_1.head(1)

,full_text,label
0,Hi piston My crankshaft depends on your bore ...,Providing_information


In [ ]:
labeled_1.full_text.iloc[1]

'How low can you go on the piston bore diameter ? '

In [ ]:
input_data = "/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/bert_aug/train_Q2.tsv"
output_file = "/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/bert_aug/distilbert_train_Q2.tsv"
writer = open(output_file, 'w')
lines = open(input_data, 'r').readlines()


for i, line in enumerate(lines):
  # parts = line[:-1].split('\t')
  parts = line.split(" ",1)
  label = parts[0]
  sentence = parts[1]
  writer.write(label + "\t" + aug.augment(parts[1]) + '\n')
  print(i)

writer.close()

In [ ]:
from csv import writer

In [ ]:
# input_data = "/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/bert_aug/train_Q2.tsv"
output_file = "/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/bert_aug/bert_labeled_2.csv"
# writer = open(output_file, 'w')
# lines = open(input_data, 'r').readlines()


with open(output_file, 'a') as f_object:
    writer_object = writer(f_object)

####For labeled_1
    for i in range(labeled_2.shape[0]):
      label = labeled_2.label.iloc[i]
      sentence = labeled_2.full_text.iloc[i]
      writer_object.writerow([aug.augment(sentence), label])
      writer
      print(i)


In [ ]:
aug_labeled_1 = pd.read_csv('/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/bert_aug/bert_labeled_1.csv', header = None)
aug_labeled_2 = pd.read_csv('/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/bert_aug/bert_labeled_2.csv', header = None)

aug_labeled_1.columns = ['full_text', 'label']
aug_labeled_2.columns = ['full_text', 'label']

In [ ]:
with open('/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/bert_aug/bert_labeled_1.pkl', 'wb') as f:
        pickle.dump(aug_labeled_1, f)

with open('/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/bert_aug/bert_labeled_2.pkl', 'wb') as f:
        pickle.dump(aug_labeled_2, f)

In [ ]:
###test if pickle files were saved properly

with open('/content/drive/MyDrive/Capstone-project_Bhavika/data_aug/bert_aug/bert_labeled_2.pkl', 'rb') as t:
    bert_labeled_2 = pickle.load(t)
bert_labeled_2

,full_text,label
0,how low can pitch you go off on the piston bor...,Exploration_of_design_parameter_values
1,all my factorofsafety values tested are in abo...,Effects_of_design_parameters_on_objectives
2,1 hi flywheel shaft my crankshaft design depen...,Dependencies_between_design_parameters
3,min as of 40,Selected_design_parameter_values_for_objective(s)
4,what value works that best if for your simulat...,Exploration_of_design_parameter_values
...,...,...
223,we are at this excellent agian,Monitoring_objective_values
224,flywheel changed a value now and i once lost 5...,Effects_of_design_parameters_on_objectives
225,now i have decreased my weight parameter a bit,Effects_of_design_parameters_on_objectives
226,bearingoffset equals 1 to 0 60 pa dc equals to 15,Dependencies_between_design_parameters


Concluion: We found out that the best augmentation results were given out by m ethod 3. Hence, we used the augmented data given out by method 3 - i.e. by feeding surroundings word to BERT language model to find out the most suitlabe word for augmentation and moved on to do further steps.